<a href="https://colab.research.google.com/github/MinsooKwak/Kaggle_note/blob/main/Data_science/yoga_posture_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

참고 링크: https://www.kaggle.com/code/vencerlanz09/yoga-pose-classification-using-cnn-mobilenetv3

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import warnings
warnings.filterwarnings('ignore')

# import Libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import itertools
import random

# import visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm  # 폰트 관련 용도
import matplotlib.cm as cm
import cv2

# Tensorflow Libraries
from tensorflow import keras
from tensorflow.keras import layers, models
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import Model
from tensorflow.keras.layers.experimental import preprocessing

# System libraries
from pathlib import Path
import os.path

# Metrics
from sklearn.metrics import classification_report, confusion_matrix

**confusion matrix**

https://jerimo.github.io/python/confusion-matrix/

In [7]:
# Wget으로 파일 다운(helper_functions.py)
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import series of helper functions for our notebook 
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, compare_historys, pred_and_plot

--2022-09-13 15:21:00--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-09-13 15:21:00 (73.1 MB/s) - ‘helper_functions.py’ saved [10246/10246]



### 한글 폰트 설치 및 설정

In [ ]:
#!sudo apt-get install -y fonts-nanum  ## 폰트설치
#!sudo fc-cache -fv ## 폰트 업데이트
#!rm ~/.cache/matplotlib -rf ## 캐쉬된 matplotlib 폰트 삭제하여 재로딩

In [ ]:
# ttflist에 Nanum 글자 들어가는 폰트 있는지 확인하는 코드
f = [f.name for f in fm.fontManager.ttflist if 'Nanum' in f.name]
f

In [ ]:
#폰트종류설정
plt.rcParams["font.family"] ='NanumSquare'

# 현재 설정되어 있는 폰트 사이즈와 글꼴을 알아보자
!python --version
def current_font():
  print(f"설정 폰트 글꼴: {plt.rcParams['font.family']}, 설정 폰트 사이즈: {plt.rcParams['font.size']}")  # 파이썬 3.6 이상 사용가능하다
        
current_font()

In [ ]:
path = '/content/drive/MyDrive/text/NanumSquareR.ttf'  # 설치된 나눔글꼴중 원하는 녀석의 전체 경로를 가져오자
font_name = fm.FontProperties(fname=path, size=10).get_name()
print(font_name)
plt.rc('font', family=font_name)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import font_manager
plt.rcParams['axes.unicode_minus']=False
font_path = '/content/drive/MyDrive/text/NanumSquareR.ttf'
fontprop = font_manager.FontProperties(fname=font_path, size=20)

plt.rc('font', family='NanumBarunGothic')

# Load and transform Data

In [8]:
BATCH_SIZE = 32
IMAGE_SIZE = (224,224)

In [9]:
dataset = "/content/drive/MyDrive/kaggle/yoga_posture_dataset"

def walk_through_dir(dir_path):
  '''
  Args:
    dir_path (str): target directory
  
  Returns:
    number of subdirectories in dir_path
    number of image(files) in each subdirectory
    name of each subdirectory
  '''
  for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

walk_through_dir(dataset)

There are 47 directories and 1 images in '/content/drive/MyDrive/kaggle/yoga_posture_dataset'.
There are 0 directories and 59 images in '/content/drive/MyDrive/kaggle/yoga_posture_dataset/Ardha Chandrasana'.
There are 0 directories and 81 images in '/content/drive/MyDrive/kaggle/yoga_posture_dataset/Baddha Konasana'.
There are 0 directories and 13 images in '/content/drive/MyDrive/kaggle/yoga_posture_dataset/Ardha Navasana'.
There are 0 directories and 74 images in '/content/drive/MyDrive/kaggle/yoga_posture_dataset/Adho Mukha Svanasana'.
There are 0 directories and 54 images in '/content/drive/MyDrive/kaggle/yoga_posture_dataset/Ardha Pincha Mayurasana'.
There are 0 directories and 12 images in '/content/drive/MyDrive/kaggle/yoga_posture_dataset/Ashta Chandrasana'.
There are 0 directories and 71 images in '/content/drive/MyDrive/kaggle/yoga_posture_dataset/Anjaneyasana'.
There are 0 directories and 18 images in '/content/drive/MyDrive/kaggle/yoga_posture_dataset/Alanasana'.
There are 

-> 해당 내용 보기 쉽게 수정할 필요 있을 것 같음